In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import utils as uti

sys.path.append(workdir_ + 'PyRegridding/Utils/')
import MakePressures as MkP
import GridUtils as GrU


sys.path.append(workdir_ + 'PyRegridding/Drivers/')
import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
### Make a list of files

In [ ]:
%%time

exp='c64_gwR2_ne30pg3_FMTHIST_L120_topfix_oldCTL'
subd, Src, Hkey='hist','ne30pg3','c'
useri='juliob'
Dst='fv1x1'
A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )

exp_A=A.exp
exp_Val='ERA5'


In [ ]:
%%time
exp='c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL_qbotune_x03'
subd, Src, Hkey='hist','ne30pg3','c'
useri='juliob'
Dst='fv1x1'

B = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )

exp_B=B.exp



In [ ]:
%%time

exp='c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL'
#exp='c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL_qbotune_x02'
subd, Src, Hkey='hist','ne30pg3','c'
useri='juliob'
Dst='fv1x1'

C = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )

In [ ]:
########################################
# Extract U
########################################

print( f'{A.exp} Years {len(A.X.time) /12. }')
UUa_s=A.X.U.values

print( f'{B.exp} Years {len(B.X.time) /12. }')
UUb_s=B.X.U.values

print( f'{C.exp} Years {len(C.X.time) /12. }')
UUc_s=C.X.U.values


In [ ]:
########################################
# Regrid
########################################

if (Hkey == 'yx' ):
    ########################
    # Option 1 from lat-lon regirrdded data
    ########################
    
    
    
    #ps_x =Dx.PS.values
    lon_a=A.X.lon.values
    lat_a=A.X.lat.values
    lev_a=A.X.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )
    # Zonal Av
    UUa_sz=np.average( UUa_s , axis=3 )

elif (Hkey=='c'):
    ########################
    # Option 2 from unstructured SE data
    ########################
    
    
    DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
    lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
    lev_a=A.X.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )
    
    RegridObj = RgF.Horz(Src=Src, Dst=Dst )
    UUa_s_xy=RgF.Horz(xfld_Src=UUa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
   
    # Zonal Av
    UUa_sz=np.average( UUa_s_xy , axis=3 )


In [ ]:
########################################
# Regrid
########################################

if (Hkey == 'yx' ):
    ########################
    # Option 1 from lat-lon regirrdded data
    ########################
    
    
    
    #ps_x =Dx.PS.values
    lon_c=C.X.lon.values
    lat_c=C.X.lat.values
    lev_c=C.X.lev.values
    zlev_c=-7.0*np.log( lev_b/1_000. )
    # Zonal Av
    UUc_sz=np.average( UUc_s , axis=3 )

elif (Hkey=='c'):
    ########################
    # Option 2 from unstructured SE data
    ########################
    
    
    DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
    lat_c,lon_c = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
    lev_c=C.X.lev.values
    zlev_c=-7.0*np.log( lev_c/1_000. )
    
    RegridObj = RgF.Horz(Src=Src, Dst=Dst )
    UUc_s_xy=RgF.Horz(xfld_Src=UUc_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
   
    # Zonal Av
    UUc_sz=np.average( UUc_s_xy , axis=3 )


In [ ]:
########################################
# Regrid
########################################

if (Hkey == 'yx' ):
    ########################
    # Option 1 from lat-lon regirrdded data
    ########################
    
    
    
    #ps_x =Dx.PS.values
    lon_a=B.X.lon.values
    lat_a=B.X.lat.values
    lev_a=B.X.lev.values
    zlev_b=-7.0*np.log( lev_b/1_000. )
    # Zonal Av
    UUb_sz=np.average( UUb_s , axis=3 )

elif (Hkey=='c'):
    ########################
    # Option 2 from unstructured SE data
    ########################
    
    
    DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
    lat_b,lon_b = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
    lev_b=B.X.lev.values
    zlev_b=-7.0*np.log( lev_b/1_000. )
    
    RegridObj = RgF.Horz(Src=Src, Dst=Dst )
    UUb_s_xy=RgF.Horz(xfld_Src=UUb_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
   
    # Zonal Av
    UUb_sz=np.average( UUb_s_xy , axis=3 )


In [ ]:
print(UUa_s_xy.shape)
#UUa_sz=np.average( UUa_s_xy , axis=3 )

In [ ]:
%%time
####
# Validation
####

ERA5dir = '/glade/campaign/cgd/amp/juliob/ERA5/Zonal_monthly/'
# get ERA5 pl 
path_C = ERA5dir + 'ERA5.native.time.*.nc'
Dc = xr.open_mfdataset( path_C ,data_vars='different', coords='different' )
lev_ERA = 1_000.*Dc.hyam.values[:] +1_000.*Dc.hybm.values[:] 
lat_ERA = Dc.latitude.values
zlev_ERA = -7. * np.log( lev_ERA /1_000. )
mos_ERA=1979.+np.arange( 528 )/12.
Uval=Dc.U.values

In [ ]:
%%time
####
# Validation
####

ERA5dir = '/glade/campaign/cgd/amp/juliob/ERA5/Zonal_monthly/'
# get ERA5 pl 
path_C = ERA5dir + 'ERA5.native.time.*.nc'
Dc = xr.open_mfdataset( path_C ) #, coords='different' )
lev_ERA = 1_000.*Dc.hyam.values[0,:] +1_000.*Dc.hybm.values[0,:] 
lat_ERA = Dc.latitude.values
zlev_ERA = -7. * np.log( lev_ERA /1_000. )
mos_ERA=1979.+np.arange( 528 )/12.
Uval=Dc.U.values

In [ ]:
print( type( A.X.date[0].values.item() ) )
print( int(( A.X.date[0].values)/10_000  ))



In [ ]:

print( np.shape(Uval) )
print( np.shape(lat_ERA) )
print( np.shape(zlev_ERA) )


In [ ]:
yearA_0 = int(( A.X.date[0].values)/10_000  )
yearB_0 = int(( B.X.date[0].values)/10_000  )
yearC_0 = int(( C.X.date[0].values)/10_000  )


In [ ]:
nmo_a, nz_a, ny_a = UUa_sz.shape
mos_a=np.arange(nmo_a)/12.  +  yearA_0

nmo_b, nz_b, ny_b = UUb_sz.shape
mos_b=np.arange(nmo_b)/12.+ yearB_0

nmo_c, nz_c, ny_c = UUc_sz.shape
mos_c=np.arange(nmo_c)/12.+ yearC_0

mos_ERA=1979.+np.arange( 528 )/12.
qbolv=np.linspace(-30,30,11)

In [ ]:
fig = plt.figure(figsize=(25, 5))
cmap='gist_ncar'
qbolv=np.linspace(-35,25,21)

nxplo=4
nyplo=1

n=2
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )

co1=ax1.contourf(mos_b,zlev_b,UUb_sz[:,:,96].T,levels=qbolv, cmap=cmap)
cb=plt.colorbar(co1)
ax1.set_xlim(yearB_0 ,yearB_0+12)
ax1.set_title(f"{B.exp} \n effgw_beres_dp=0.4")

n=3
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )

co1=ax1.contourf(mos_a,zlev_a,UUa_sz[:,:,96].T,levels=qbolv, cmap=cmap)
cb=plt.colorbar(co1)
ax1.set_xlim(yearA_0 ,yearA_0+12)
ax1.set_title(f"{A.exp} \n effgw_beres_dp=0.4")

n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )

co1=ax1.contourf(mos_c,zlev_c,UUc_sz[:,:,96].T,levels=qbolv, cmap=cmap)
cb=plt.colorbar(co1)
ax1.set_xlim(yearC_0 ,yearC_0+12)
ax1.set_title(f"{C.exp} \n effgw_beres_dp=0.7")

n=4
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(mos_ERA,zlev_ERA,Uval[:,:,320].T,levels=qbolv, cmap=cmap)
cb=plt.colorbar(co1)
ax1.set_xlim(1985,1997)
ax1.set_title(f"ERA5")


In [ ]:
plt.contourf(mos_a,zlev_a,UUa_sz[:,:,96].T,levels=qbolv, cmap=cmap)
plt.colorbar()
plt.xlim(1985,1997)
#plt.xlim(0,1000)

In [ ]:
plt.contourf(mos_b,zlev_b,UUb_sz[:,:,96].T,levels=qbolv, cmap=cmap)
plt.colorbar()
plt.xlim(1985,1997)
#plt.xlim(0,1000)

In [ ]:
plt.contourf(mos_ERA,zlev_ERA,Uval[:,:,320].T,levels=qbolv, cmap=cmap)
plt.xlim(1985,1997)
plt.colorbar()